In [2]:
import pandas as pd
#from google.colab import files
#import io
#Para subir los archivos
#uploaded = files.upload()

In [3]:
df_2 = pd.read_excel('df_2.xlsx')
df_2.head()
#La variable a predecir es la p_matematica

,Dist_cen,M_Dist_cen,p_matematica,genero,nivel_educacion_madre,quintil_income,Num_docentes,capacidad_uso,edad
0,0.669,0.770,14.0,1,3,1,39,0.8357,12.568127
1,0.834,1.126,17.0,0,5,3,53,0.9010,12.568127
2,1.794,2.502,14.0,1,3,2,17,0.5000,9.450796
3,0.980,1.386,11.0,1,3,1,52,0.7475,10.467105
4,2.317,2.680,12.0,0,3,2,16,0.5371,11.484286


In [5]:
df_2['nivel_educacion_madre_c'] = df_2['nivel_educacion_madre'].astype('category')
df_2['quintil_income_c'] = df_2['quintil_income'].astype('category')
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9201 entries, 0 to 9200
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Dist_cen                 9201 non-null   float64 
 1   M_Dist_cen               9201 non-null   float64 
 2   p_matematica             9201 non-null   float64 
 3   genero                   9201 non-null   int64   
 4   nivel_educacion_madre    9201 non-null   int64   
 5   quintil_income           9201 non-null   int64   
 6   Num_docentes             9201 non-null   int64   
 7   capacidad_uso            9201 non-null   float64 
 8   edad                     9201 non-null   float64 
 9   nivel_educacion_madre_c  9201 non-null   category
 10  quintil_income_c         9201 non-null   category
dtypes: category(2), float64(5), int64(4)
memory usage: 665.4 KB


In [6]:
df_2.drop(['nivel_educacion_madre','quintil_income'], axis=1, inplace=True) #axis = 0 es fila, 1 es columna
df_2.head()

,Dist_cen,M_Dist_cen,p_matematica,genero,Num_docentes,capacidad_uso,edad,nivel_educacion_madre_c,quintil_income_c
0,0.669,0.770,14.0,1,39,0.8357,12.568127,3,1
1,0.834,1.126,17.0,0,53,0.9010,12.568127,5,3
2,1.794,2.502,14.0,1,17,0.5000,9.450796,3,2
3,0.980,1.386,11.0,1,52,0.7475,10.467105,3,1
4,2.317,2.680,12.0,0,16,0.5371,11.484286,3,2


In [7]:
df_Ml = pd.get_dummies(df_2, prefix = {'nivel_educacion_madre_c' : 'nem', 'quintil_income_c' : 'qin'}, drop_first=True) #El drop mata 1 dummy
df_Ml.head()
#Se recomienda el uso de dummy debido a los problemas de escala

,Dist_cen,M_Dist_cen,p_matematica,genero,Num_docentes,capacidad_uso,edad,nem_2,nem_3,nem_4,nem_5,qin_2,qin_3
0,0.669,0.770,14.0,1,39,0.8357,12.568127,0,1,0,0,0,0
1,0.834,1.126,17.0,0,53,0.9010,12.568127,0,0,0,1,0,1
2,1.794,2.502,14.0,1,17,0.5000,9.450796,0,1,0,0,1,0
3,0.980,1.386,11.0,1,52,0.7475,10.467105,0,1,0,0,0,0
4,2.317,2.680,12.0,0,16,0.5371,11.484286,0,1,0,0,1,0


In [8]:
from sklearn.preprocessing import MinMaxScaler
cols_to_scale = ['Dist_cen','M_Dist_cen','Num_docentes','edad','p_matematica'] #Poner todas las va a normalizar
scaler = MinMaxScaler()
df_Ml[cols_to_scale] = scaler.fit_transform(df_Ml[cols_to_scale])
df_Ml.head()

,Dist_cen,M_Dist_cen,p_matematica,genero,Num_docentes,capacidad_uso,edad,nem_2,nem_3,nem_4,nem_5,qin_2,qin_3
0,0.018217,0.014727,0.466667,1,0.430556,0.8357,0.845055,0,1,0,0,0,0
1,0.030899,0.039458,0.566667,0,0.625000,0.9010,0.845055,0,0,0,1,0,1
2,0.104689,0.135047,0.466667,1,0.125000,0.5000,0.421433,0,1,0,0,1,0
3,0.042121,0.057520,0.366667,1,0.611111,0.7475,0.559542,0,1,0,0,0,0
4,0.144889,0.147412,0.400000,0,0.111111,0.5371,0.697769,0,1,0,0,1,0


In [9]:
#Separamos entre variables dependientes y independientes
Dep_vars = ['p_matematica']
x = df_Ml.drop(Dep_vars, axis = 'columns')
y = df_Ml['p_matematica']

In [10]:
#Entrenamiento y testeo
from sklearn.model_selection import train_test_split #Libreria para separar la data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=20) #el random_state = semilla de rng